In [ ]:


import pandas as pd
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# Define the dataset path
file_path = r"C:\Users\Haji\Documents\muzamil data\swat.csv"

# Load the dataset
df = pd.read_csv(file_path)

In [ ]:

# Load the dataset
file_path = r"C:\Users\Haji\Documents\muzamil data\swat.csv"
df = pd.read_csv(file_path)

# Check for missing values
df = df.dropna()

# Separate features and target
X = df.drop(columns=['label'])
y = df['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.6, random_state=42, stratify=y)

# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

# Train and evaluate models
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix for {name}")
    plt.show()


In [ ]:
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
# Separate features and target
X = df.drop(columns=['label'])
y = df['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply Isolation Forest
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
iso_preds = iso_forest.fit_predict(X_scaled)

# Convert Isolation Forest output to binary (1: normal, -1: anomaly)
iso_preds = np.where(iso_preds == -1, 1, 0)

# Stack Isolation Forest predictions with original features
X_hybrid = np.column_stack((X_scaled, iso_preds))

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_hybrid, y, test_size=0.2, random_state=42, stratify=y)

# Train XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Predictions
y_pred = xgb_model.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Hybrid Isolated+XGBoost Model Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

In [ ]:
# Separate features and target
X = df.drop(columns=['label'])
y = df['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Define deep learning models
# 1. Fully Connected Neural Network (MLP)
def build_mlp():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 2. Convolutional Neural Network (CNN)
def build_cnn():
    model = keras.Sequential([
        keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
        keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 3. Long Short-Term Memory (LSTM)
def build_lstm():
    model = keras.Sequential([
        keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
        keras.layers.LSTM(50, return_sequences=True),
        keras.layers.LSTM(25),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate each model
models = {"MLP": build_mlp(), "CNN": build_cnn(), "LSTM": build_lstm()}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

In [ ]:
# Separate features and target
X = df.drop(columns=['label'])
y = df['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# 1. Autoencoder for feature extraction
def build_autoencoder():
    input_dim = X_train.shape[1]
    autoencoder = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(input_dim, activation='sigmoid')
    ])
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

autoencoder = build_autoencoder()
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test), verbose=1)
X_train_enc = autoencoder.predict(X_train)
X_test_enc = autoencoder.predict(X_test)

# 2. Isolation Forest
def apply_isolation_forest(X_train_enc, X_test_enc):
    iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
    iso_preds_train = iso_forest.fit_predict(X_train_enc)
    iso_preds_test = iso_forest.predict(X_test_enc)
    iso_preds_train = np.where(iso_preds_train == -1, 1, 0)
    iso_preds_test = np.where(iso_preds_test == -1, 1, 0)
    return iso_preds_train, iso_preds_test

iso_preds_train, iso_preds_test = apply_isolation_forest(X_train_enc, X_test_enc)

# Stack features with isolation forest predictions
X_train_hybrid = np.column_stack((X_train_enc, iso_preds_train))
X_test_hybrid = np.column_stack((X_test_enc, iso_preds_test))

# 3. Train Hybrid Model (XGBoost + Random Forest + LSTM)

# XGBoost Model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_hybrid, y_train)
y_pred_xgb = xgb_model.predict(X_test_hybrid)

# Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_hybrid, y_train)
y_pred_rf = rf_model.predict(X_test_hybrid)

# LSTM Model
def build_lstm():
    model = keras.Sequential([
        keras.layers.Reshape((X_train_hybrid.shape[1], 1), input_shape=(X_train_hybrid.shape[1],)),
        keras.layers.LSTM(50, return_sequences=True),
        keras.layers.LSTM(25),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

lstm_model = build_lstm()
lstm_model.fit(X_train_hybrid, y_train, epochs=10, batch_size=32, validation_data=(X_test_hybrid, y_test), verbose=1)
y_pred_lstm = (lstm_model.predict(X_test_hybrid) > 0.5).astype(int)

# Final Predictions (Majority Voting)
final_preds = np.round((y_pred_xgb + y_pred_rf + y_pred_lstm.flatten()) / 3)

# Evaluation
accuracy = accuracy_score(y_test, final_preds)
print("\nHybrid Model (Autoencoder + Isolation Forest + XGBoost + Random Forest + LSTM) Results:")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, final_preds))